In [2]:
import pandas as pd
import numpy as np

import os
from time import gmtime, strftime, sleep
import json

import boto3
import sagemaker
import sagemaker.session

from sagemaker import ModelPackage
from time import gmtime, strftime
from sagemaker.predictor import json_serializer, json_deserializer, Predictor

In [3]:
session = sagemaker.session.Session()
# pipe_session= PipelineSession()
region = session.boto_region_name
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
account_id = session.account_id()
client = boto3.client('sagemaker', region_name=region)

tags = [
    {"Key": "PLATFORM", "Value": "FO-ML"},
    {"Key": "BUSINESS_REGION", "Value": "GLOBAL"},
    {"Key": "BUSINESS_UNIT", "Value": "MOBILITY"},
    {"Key": "CLIENT", "Value": "MULTI_TENANT"}
   ]

timestamp_suffix = strftime("%Y-%m-%d-%H-%M", gmtime())
endpoint_name = 'predictionBugs-' + timestamp_suffix

model_package_group_name = 'mlpipe'
version = 1
model_package_arn = f'arn:aws:sagemaker:{region}:{account_id}:model-package/{model_package_group_name}/{version}'

In [4]:
model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=session)
model.deploy(initial_instance_count=1,
             instance_type='ml.m5.xlarge',
             endpoint_name = endpoint_name,
             tags=tags)

-------------------!

In [5]:
payload = json.dumps('{"true_false":null,"one_hot":"yellow","dates":1671062400000,"floats":9.0,"max_of_list":"4,3,9,6,0","nunique_of_list":"4,9","desc_stats":"9,4","multi_label":"strawberry","random_col":1.0,"other":3}')

In [6]:
payload

'"{\\"true_false\\":null,\\"one_hot\\":\\"yellow\\",\\"dates\\":1671062400000,\\"floats\\":9.0,\\"max_of_list\\":\\"4,3,9,6,0\\",\\"nunique_of_list\\":\\"4,9\\",\\"desc_stats\\":\\"9,4\\",\\"multi_label\\":\\"strawberry\\",\\"random_col\\":1.0,\\"other\\":3}"'

In [7]:
predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=session,
                      serializer=json_serializer,
                      content_type='application/json',
                      accept='application/json')

print(predictor.predict(payload))

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
accept is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The json_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from container-1 with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/predictionBugs-2023-05-19-15-26 in account 707031497630 for more information.

![error logs](error_logs.png)

# Delete endpoint

In [9]:
predictor.delete_endpoint()